In [8]:
# import numpy as np
# import tensorflow as tf
# from keras.models import load_model
# import pandas as pd
# import time

# # Timestamp helper
# def timestamp(): return time.strftime('%Y-%m-%d %H:%M:%S')

# # Parameters for inference
# original_samples = 20000
# seq_length = 20
# original_steps_to_pred = 12000
# batch_size = 32
# molecules = 2
# species = '2_22'
# horizons = np.array([1,5,10,20])
# downsample_factor = 1

# # 🔥 NEW: Interval reporting configuration
# REPORT_INTERVAL = 500  # Report MAPE every 500 steps
# MILESTONE_INTERVALS = [1000, 2500, 5000, 10000, 25000]  # Special milestone reports

# samples = original_samples
# steps_to_pred = original_steps_to_pred

# print(f"[{timestamp()}] 📊 Original samples: {original_samples}, Using: {samples}")
# print(f"[{timestamp()}] 📊 Steps to predict: {steps_to_pred}")
# print(f"[{timestamp()}] 📊 Reporting MAPE every {REPORT_INTERVAL} steps")

# # Load the data
# data = np.load('%s_data.npy' % species)
# print(f"[{timestamp()}] 📁 Loaded data with shape: {data.shape}")

# data = data[:samples, :]
# print(f"[{timestamp()}] 📁 Trimmed data shape: {data.shape}")

# # Verify binary flags
# binary_flags = data[:, 6]
# ones_indices = np.where(binary_flags == 1)[0]
# print(f"[{timestamp()}] ⚡ Found {len(ones_indices)} instances of 1 in binary flag")
# if len(ones_indices) > 0:
#     print(f"First 10 indices with 1: {ones_indices[:10]}")

# # Prepare test data
# test_data = data[-steps_to_pred - seq_length:, :]
# print(f"[{timestamp()}] 🧪 Test data shape: {test_data.shape}")

# # Load the model
# model = load_model('transformer_model_384_%s.h5' % species)
# print(f"[{timestamp()}] ✅ Loaded model")
# print(f"Model input shape: {model.input_shape}")
# print(f"Model output shape: {model.output_shape}")

# # Helper functions
# def create_initial_test_sequence(test_data, seq_length, step_index=0):
#     sequence = test_data[step_index:step_index + seq_length, :]
#     return np.expand_dims(sequence, axis=0)

# def get_ground_truth(test_data, seq_length, step_index):
#     truth = test_data[step_index + seq_length, :6]
#     return np.expand_dims(truth, axis=0)

# # 🔥 NEW: Function to calculate and display interval statistics
# def report_interval_stats(performance, current_step, interval_size, mode, start_time):
#     """Report MAPE statistics for the current interval and overall progress"""
    
#     # Get valid performance data up to current step
#     valid_perf = performance[:current_step+1][performance[:current_step+1, 0] >= 0, 0]
    
#     if len(valid_perf) == 0:
#         print(f"[{timestamp()}] Step {current_step+1:5d}: No valid predictions yet")
#         return
    
#     # Overall statistics
#     overall_avg = np.mean(valid_perf)
#     overall_std = np.std(valid_perf)
#     overall_median = np.median(valid_perf)
    
#     # Interval statistics (last 'interval_size' valid predictions)
#     interval_data = valid_perf[-interval_size:] if len(valid_perf) >= interval_size else valid_perf
#     interval_avg = np.mean(interval_data)
#     interval_std = np.std(interval_data)
    
#     # Calculate speed and ETA
#     elapsed_time = time.time() - start_time
#     steps_per_second = (current_step + 1) / elapsed_time if elapsed_time > 0 else 0
#     remaining_steps = steps_to_pred - (current_step + 1)
#     eta_seconds = remaining_steps / steps_per_second if steps_per_second > 0 else 0
#     eta_minutes = eta_seconds / 60
    
#     # Format ETA
#     if eta_minutes > 60:
#         eta_str = f"ETA: {eta_minutes/60:.1f}h"
#     else:
#         eta_str = f"ETA: {eta_minutes:.1f}min"
    
#     # Print comprehensive report
#     print(f"[{timestamp()}] Step {current_step+1:5d}/{steps_to_pred} | Mode: {mode:13s}")
#     print(f"    📊 Overall MAPE:  {overall_avg:.4f} ± {overall_std:.4f} | Median: {overall_median:.4f}")
#     print(f"    📈 Last {len(interval_data):3d} MAPE: {interval_avg:.4f} ± {interval_std:.4f}")
#     print(f"    ⚡ Speed: {steps_per_second:.1f}/s | Valid: {len(valid_perf):5d}/{current_step+1} | {eta_str}")

# # 🔥 NEW: Function for milestone reporting
# def report_milestone(performance, current_step, milestone_windows=[100, 500, 1000]):
#     """Report MAPE for different time windows"""
    
#     valid_perf = performance[:current_step+1][performance[:current_step+1, 0] >= 0, 0]
    
#     if len(valid_perf) < 50:  # Need minimum data for meaningful statistics
#         return
    
#     print(f"    🎯 MILESTONE {current_step+1:5d}:")
    
#     for window in milestone_windows:
#         if len(valid_perf) >= window:
#             window_data = valid_perf[-window:]
#             window_avg = np.mean(window_data)
#             print(f"       Last {window:4d}: {window_avg:.4f}")
#         else:
#             if len(valid_perf) > 10:  # Only show if we have some data
#                 window_avg = np.mean(valid_perf)
#                 print(f"       All {len(valid_perf):4d}: {window_avg:.4f}")
#             break

# # 🔥 NEW: Function to detect MAPE trends
# def detect_trend_alerts(performance, current_step, alert_window=100):
#     """Detect if MAPE is trending up or down"""
    
#     if current_step < alert_window * 2:
#         return
    
#     valid_perf = performance[:current_step+1][performance[:current_step+1, 0] >= 0, 0]
    
#     if len(valid_perf) < alert_window * 2:
#         return
    
#     # Compare recent vs earlier performance
#     recent_avg = np.mean(valid_perf[-alert_window:])
#     earlier_avg = np.mean(valid_perf[-alert_window*2:-alert_window])
    
#     change_pct = (recent_avg - earlier_avg) / earlier_avg * 100
    
#     if abs(change_pct) > 10:  # Alert if >10% change
#         trend = "IMPROVING" if change_pct < 0 else "DEGRADING"
#         print(f"    🚨 TREND ALERT: MAPE {trend} by {abs(change_pct):.1f}% over last {alert_window} steps")

# # Track performance summary
# summary = {}

# # Loop over horizons
# for j in range(len(horizons)):
#     horizon = horizons[j]
#     horizon_start_time = time.time()
    
#     print(f"\n[{timestamp()}] 🚀 Starting inference for horizon {horizon}")
#     print("="*80)

#     performance = np.zeros((steps_to_pred, 1))
#     predictions = np.zeros((steps_to_pred, 2 * molecules * 3))
#     test = create_initial_test_sequence(test_data, seq_length)
#     using_ground_truth = True

#     for i in range(steps_to_pred):
#         prediction = model.predict(test, steps=1, verbose=0)

#         if i + seq_length < len(test_data):
#             ground_truth = get_ground_truth(test_data, seq_length, i)
#             error = tf.metrics.mean_absolute_percentage_error(ground_truth, prediction)
#             performance[i, 0] = error
#             predictions[i, 0:molecules * 3] = prediction
#             predictions[i, molecules * 3:2 * molecules * 3] = ground_truth
#         else:
#             print(f"[{timestamp()}] ⚠️ Step {i} exceeds available test data")
#             performance[i, 0] = -1
#             predictions[i, 0:molecules * 3] = prediction

#         # Track flag
#         start_index = len(data) - steps_to_pred - seq_length
#         current_index = start_index + i + seq_length
#         binary_flag_value = data[current_index, 6]

#         full_prediction = np.zeros((1, 7))
#         full_prediction[0, :6] = prediction
#         full_prediction[0, 6] = binary_flag_value

#         # Horizon handling
#         if horizon != 1:
#             if i % horizon != 0:
#                 full_prediction_reshaped = np.expand_dims(full_prediction, axis=1)
#                 test = np.append(test[:, 1:, :], full_prediction_reshaped, axis=1)
#                 if using_ground_truth:
#                     print(f"[{timestamp()}] Step {i}: 🔁 Switching to auto-regressive prediction")
#                     using_ground_truth = False
#             else:
#                 if i + seq_length + 1 < len(test_data):
#                     test = create_initial_test_sequence(test_data, seq_length, i + 1)
#                     print(f"[{timestamp()}] Step {i}: 🔄 Resetting with ground truth")
#                     using_ground_truth = True
#                 else:
#                     full_prediction_reshaped = np.expand_dims(full_prediction, axis=1)
#                     test = np.append(test[:, 1:, :], full_prediction_reshaped, axis=1)
#                     print(f"[{timestamp()}] Step {i}: Continuing auto-regressive (beyond ground truth)")
#                     using_ground_truth = False
#         else:
#             if i + seq_length + 1 < len(test_data):
#                 test = create_initial_test_sequence(test_data, seq_length, i + 1)
#                 using_ground_truth = True
#             else:
#                 full_prediction_reshaped = np.expand_dims(full_prediction, axis=1)
#                 test = np.append(test[:, 1:, :], full_prediction_reshaped, axis=1)
#                 print(f"[{timestamp()}] Step {i}: Switching to auto-regressive (beyond ground truth)")
#                 using_ground_truth = False

#         # 🔥 ENHANCED REPORTING: Multiple reporting triggers
#         mode = "Ground Truth" if using_ground_truth else "Auto-Regressive"
        
#         # 1. Regular interval reporting
#         if (i + 1) % REPORT_INTERVAL == 0 or i == steps_to_pred - 1:
#             report_interval_stats(performance, i, REPORT_INTERVAL, mode, horizon_start_time)
        
#         # 2. Milestone reporting
#         if (i + 1) in MILESTONE_INTERVALS:
#             report_milestone(performance, i)
        
#         # 3. Trend detection
#         if (i + 1) % 1000 == 0:  # Check trends every 1000 steps
#             detect_trend_alerts(performance, i)
        
#         # 4. Individual step reporting for first few steps
#         if i < 10:
#             valid_perf = performance[:i+1][performance[:i+1, 0] >= 0, 0]
#             if len(valid_perf) > 0:
#                 step_mape = performance[i, 0] if performance[i, 0] >= 0 else 0
#                 print(f"    └─ Step {i:3d}: Individual MAPE = {step_mape:.4f} | Mode: {mode[:2]}")
        
#         # 5. Alert for unusually high individual MAPE
#         if i > 100 and performance[i, 0] >= 0:
#             # Calculate recent average for comparison
#             recent_valid = performance[max(0, i-50):i+1][performance[max(0, i-50):i+1, 0] >= 0, 0]
#             if len(recent_valid) > 10:
#                 recent_avg = np.mean(recent_valid)
#                 current_mape = performance[i, 0]
                
#                 if current_mape > recent_avg * 3 and current_mape > 1.0:  # Alert threshold
#                     print(f"    ⚠️  HIGH MAPE Step {i}: {current_mape:.4f} (recent avg: {recent_avg:.4f}) | Mode: {mode[:2]}")

#     print(f"\n[{timestamp()}] ✅ Completed prediction loop for horizon {horizon}")
    
#     # Save results
#     perf_file = f"11performance_{species}_horizon_{horizon}.csv"
#     pred_file = f"11predictions_{species}_horizon_{horizon}.csv"
#     print(f"[{timestamp()}] 💾 Saving results to {perf_file} and {pred_file}")
#     np.savetxt(perf_file, performance, delimiter=",")
#     np.savetxt(pred_file, predictions, delimiter=",")

#     # Final statistics for this horizon
#     valid_perf = performance[performance[:, 0] >= 0, 0]
#     if len(valid_perf) > 0:
#         avg_perf = np.mean(valid_perf)
#         std_perf = np.std(valid_perf)
#         median_perf = np.median(valid_perf)
#         min_perf = np.min(valid_perf)
#         max_perf = np.max(valid_perf)
#         horizon_time = time.time() - horizon_start_time
        
#         print(f"\n[{timestamp()}] 📊 FINAL STATISTICS for Horizon {horizon}:")
#         print(f"    Average MAPE: {avg_perf:.4f} ± {std_perf:.4f}")
#         print(f"    Median MAPE:  {median_perf:.4f}")
#         print(f"    Range:        {min_perf:.4f} - {max_perf:.4f}")
#         print(f"    Valid predictions: {len(valid_perf)}/{steps_to_pred} ({len(valid_perf)/steps_to_pred*100:.1f}%)")
#         print(f"    Total time: {horizon_time:.1f} seconds ({horizon_time/60:.1f} minutes)")
        
#         summary[horizon] = avg_perf
#     else:
#         print(f"[{timestamp()}] ❌ No valid performance data for horizon {horizon}")
#         summary[horizon] = None

# # Final summary
# print(f"\n{'='*80}")
# print(f"[{timestamp()}] 🧾 FINAL SUMMARY - Average MAPE by Horizon:")
# print(f"{'='*80}")
# for h, v in summary.items():
#     if v is not None:
#         print(f"Horizon {h:4d}: {v:.4f}")
#     else:
#         print(f"Horizon {h:4d}: No valid data")

# print(f"\n[{timestamp()}] 🎉 Inference completed for all horizons!")

# # Save summary to CSV
# summary_data = [[h, v if v is not None else -1] for h, v in summary.items()]
# summary_df = pd.DataFrame(summary_data, columns=['Horizon', 'Avg_MAPE'])
# summary_file = f"summary_{species}_enhanced.csv"
# summary_df.to_csv(summary_file, index=False)
# print(f"[{timestamp()}] 💾 Enhanced summary saved to {summary_file}")

In [6]:
# import numpy as np
# import tensorflow as tf
# from keras.models import load_model
# import pandas as pd
# import time

# def timestamp(): 
#     return time.strftime('%Y-%m-%d %H:%M:%S')

# # ============================================================================
# # CORRECTED PARAMETERS AND DATA HANDLING
# # ============================================================================

# # Key parameters
# TRAINING_SAMPLES = 8000      # Samples used for training
# TOTAL_SAMPLES = 20000        # Total samples in dataset
# seq_length = 20
# steps_to_pred = 12000        # How many steps to predict beyond training
# batch_size = 32
# molecules = 2
# species = '2_22'
# horizons = np.array([1, 5, 10, 20])

# print(f"[{timestamp()}] 🔧 CORRECTED INFERENCE SETUP:")
# print(f"   Training used: {TRAINING_SAMPLES} samples")
# print(f"   Total dataset: {TOTAL_SAMPLES} samples") 
# print(f"   Predicting: {steps_to_pred} steps beyond training")
# print(f"   Available for prediction: {TOTAL_SAMPLES - TRAINING_SAMPLES} = {TOTAL_SAMPLES - TRAINING_SAMPLES} samples")

# # ============================================================================
# # LOAD AND VERIFY DATA
# # ============================================================================

# # Load the FULL dataset
# full_data = np.load(f'{species}_data.npy')
# print(f"[{timestamp()}] 📁 Loaded full dataset: {full_data.shape}")

# # Verify we have enough data
# if TOTAL_SAMPLES > len(full_data):
#     print(f"❌ Error: Dataset only has {len(full_data)} samples, need {TOTAL_SAMPLES}")
#     exit(1)

# if TRAINING_SAMPLES + steps_to_pred > TOTAL_SAMPLES:
#     print(f"❌ Error: Training ({TRAINING_SAMPLES}) + prediction ({steps_to_pred}) > total ({TOTAL_SAMPLES})")
#     exit(1)

# # Use only the specified amount
# full_data = full_data[:TOTAL_SAMPLES, :]
# print(f"[{timestamp()}] 📁 Using dataset: {full_data.shape}")

# # Split data correctly
# training_data = full_data[:TRAINING_SAMPLES, :]          # First 8,000 for training context
# prediction_data = full_data[TRAINING_SAMPLES:, :]        # Remaining for prediction/validation

# print(f"[{timestamp()}] 🔄 Data split:")
# print(f"   Training context: {training_data.shape} (samples 0 to {TRAINING_SAMPLES-1})")
# print(f"   Prediction range: {prediction_data.shape} (samples {TRAINING_SAMPLES} to {TOTAL_SAMPLES-1})")

# # Verify binary flags in full dataset
# binary_flags = full_data[:, 6]
# ones_indices = np.where(binary_flags == 1)[0]
# print(f"[{timestamp()}] ⚡ Binary flag analysis:")
# print(f"   Total '1' flags: {len(ones_indices)}")
# print(f"   Pattern check - first 10 indices: {ones_indices[:10]}")
# print(f"   Expected pattern: every 100 steps starting from some offset")

# # Check if pattern is every 100 steps
# if len(ones_indices) > 1:
#     differences = np.diff(ones_indices)
#     print(f"   Intervals between flags: {differences[:10]} (should be mostly 100)")

# # ============================================================================
# # CORRECTED INFERENCE SETUP
# # ============================================================================

# def create_initial_sequence_from_training(training_data, seq_length):
#     """Create initial sequence from the LAST part of training data"""
#     if len(training_data) < seq_length:
#         raise ValueError(f"Training data too short: {len(training_data)} < {seq_length}")
    
#     # Take the last seq_length samples from training
#     sequence = training_data[-seq_length:, :]
#     return np.expand_dims(sequence, axis=0)

# def get_correct_binary_flag(full_data, training_samples, prediction_step):
#     """Get the correct binary flag for a given prediction step"""
#     # The actual position in the full dataset
#     actual_position = training_samples + prediction_step
    
#     if actual_position >= len(full_data):
#         # If we've gone beyond available data, extrapolate the pattern
#         # Assuming the pattern repeats every 100 steps
#         ones_indices = np.where(full_data[:, 6] == 1)[0]
#         if len(ones_indices) > 0:
#             # Find the pattern
#             pattern_start = ones_indices[0] % 100
#             flag_value = 1 if (actual_position % 100) == pattern_start else 0
#             print(f"   ⚡ Extrapolating flag for position {actual_position}: {flag_value}")
#             return flag_value
#         else:
#             return 0
    
#     return full_data[actual_position, 6]

# def get_ground_truth_if_available(full_data, training_samples, prediction_step):
#     """Get ground truth if available, otherwise return None"""
#     actual_position = training_samples + prediction_step
    
#     if actual_position < len(full_data):
#         return full_data[actual_position, :6].reshape(1, -1)
#     else:
#         return None

# # ============================================================================
# # CORRECTED INFERENCE LOOP
# # ============================================================================

# print(f"\n[{timestamp()}] 🚀 Starting corrected inference...")

# # Load model
# model = load_model(f'transformer_model_384_{species}.h5')
# print(f"[{timestamp()}] ✅ Model loaded: {model.input_shape} → {model.output_shape}")

# # Track performance summary
# summary = {}

# for j, horizon in enumerate(horizons):
#     horizon_start_time = time.time()
    
#     print(f"\n[{timestamp()}] 🎯 Starting horizon {horizon}")
#     print("="*80)
    
#     # Initialize tracking
#     performance = np.zeros((steps_to_pred, 1))
#     predictions = np.zeros((steps_to_pred, 2 * molecules * 3))
    
#     # Start with the last sequence from training data
#     current_sequence = create_initial_sequence_from_training(training_data, seq_length)
#     print(f"[{timestamp()}] 🔄 Initial sequence shape: {current_sequence.shape}")
#     print(f"   Starting from training samples {TRAINING_SAMPLES-seq_length} to {TRAINING_SAMPLES-1}")
    
#     using_ground_truth = True
    
#     for i in range(steps_to_pred):
#         # Make prediction
#         prediction = model.predict(current_sequence, steps=1, verbose=0)
        
#         # Get ground truth if available
#         ground_truth = get_ground_truth_if_available(full_data, TRAINING_SAMPLES, i)
        
#         # Calculate performance if ground truth available
#         if ground_truth is not None:
#             error = tf.metrics.mean_absolute_percentage_error(ground_truth, prediction)
#             performance[i, 0] = error.numpy()
#             predictions[i, :molecules*3] = prediction.flatten()
#             predictions[i, molecules*3:] = ground_truth.flatten()
#         else:
#             performance[i, 0] = -1  # No ground truth available
#             predictions[i, :molecules*3] = prediction.flatten()
#             if i == 0:
#                 print(f"[{timestamp()}] ⚠️  No more ground truth available from step {i}")
        
#         # Get the correct binary flag
#         binary_flag = get_correct_binary_flag(full_data, TRAINING_SAMPLES, i)
        
#         # Create full prediction (coordinates + binary flag)
#         full_prediction = np.zeros((1, 7))
#         full_prediction[0, :6] = prediction.flatten()
#         full_prediction[0, 6] = binary_flag
        
#         # Update sequence based on horizon strategy
#         if horizon == 1:
#             # Always use ground truth if available, otherwise use prediction
#             if ground_truth is not None:
#                 next_step = np.zeros((1, 7))
#                 next_step[0, :6] = ground_truth.flatten()
#                 next_step[0, 6] = binary_flag
#                 current_sequence = np.concatenate([current_sequence[:, 1:, :], 
#                                                  next_step.reshape(1, 1, -1)], axis=1)
#                 mode = "Ground Truth"
#             else:
#                 current_sequence = np.concatenate([current_sequence[:, 1:, :], 
#                                                  full_prediction.reshape(1, 1, -1)], axis=1)
#                 mode = "Auto-Regressive"
#                 using_ground_truth = False
                
#         else:
#             # Multi-step horizon strategy
#             if i % horizon == 0 and ground_truth is not None:
#                 # Reset with ground truth every 'horizon' steps
#                 reset_start = max(0, TRAINING_SAMPLES + i - seq_length + 1)
#                 reset_end = TRAINING_SAMPLES + i + 1
                
#                 if reset_end <= len(full_data):
#                     current_sequence = full_data[reset_start:reset_end, :].reshape(1, seq_length, -1)
#                     mode = "Ground Truth Reset"
#                     using_ground_truth = True
#                 else:
#                     # Continue with prediction
#                     current_sequence = np.concatenate([current_sequence[:, 1:, :], 
#                                                      full_prediction.reshape(1, 1, -1)], axis=1)
#                     mode = "Auto-Regressive"
#                     using_ground_truth = False
#             else:
#                 # Continue with previous prediction
#                 current_sequence = np.concatenate([current_sequence[:, 1:, :], 
#                                                  full_prediction.reshape(1, 1, -1)], axis=1)
#                 mode = "Auto-Regressive"
#                 using_ground_truth = False
        
#         # Progress reporting
#         if (i + 1) % 500 == 0 or i < 10:
#             actual_pos = TRAINING_SAMPLES + i
#             valid_perf = performance[:i+1][performance[:i+1, 0] >= 0, 0]
            
#             if len(valid_perf) > 0:
#                 avg_mape = np.mean(valid_perf)
#                 print(f"[{timestamp()}] Step {i+1:5d}/{steps_to_pred} | Pos: {actual_pos:5d} | "
#                       f"MAPE: {avg_mape:.4f} | Flag: {binary_flag} | Mode: {mode}")
#             else:
#                 print(f"[{timestamp()}] Step {i+1:5d}/{steps_to_pred} | Pos: {actual_pos:5d} | "
#                       f"No valid MAPE | Flag: {binary_flag} | Mode: {mode}")
    
#     # Save results
#     perf_file = f"corrected_performance_{species}_horizon_{horizon}.csv"
#     pred_file = f"corrected_predictions_{species}_horizon_{horizon}.csv"
#     np.savetxt(perf_file, performance, delimiter=",")
#     np.savetxt(pred_file, predictions, delimiter=",")
    
#     # Calculate final statistics
#     valid_perf = performance[performance[:, 0] >= 0, 0]
#     if len(valid_perf) > 0:
#         avg_perf = np.mean(valid_perf)
#         print(f"\n[{timestamp()}] ✅ Horizon {horizon} completed:")
#         print(f"   Average MAPE: {avg_perf:.4f}")
#         print(f"   Valid predictions: {len(valid_perf)}/{steps_to_pred}")
#         summary[horizon] = avg_perf
#     else:
#         print(f"\n[{timestamp()}] ❌ Horizon {horizon}: No valid predictions")
#         summary[horizon] = None

# # Final summary
# print(f"\n{'='*80}")
# print(f"[{timestamp()}] 🎉 CORRECTED INFERENCE COMPLETED")
# print(f"{'='*80}")
# for h, v in summary.items():
#     if v is not None:
#         print(f"Horizon {h:4d}: {v:.4f} MAPE")
#     else:
#         print(f"Horizon {h:4d}: No valid data")

# print(f"\n[{timestamp()}] 💡 Key corrections made:")
# print(f"   ✅ Start prediction from end of training data (sample {TRAINING_SAMPLES})")
# print(f"   ✅ Use correct binary flags from proper temporal positions")
# print(f"   ✅ Handle ground truth availability correctly")
# print(f"   ✅ Maintain proper sequence continuity")

[2025-07-06 19:03:16] 🔧 CORRECTED INFERENCE SETUP:
   Training used: 8000 samples
   Total dataset: 20000 samples
   Predicting: 12000 steps beyond training
   Available for prediction: 12000 = 12000 samples
[2025-07-06 19:03:16] 📁 Loaded full dataset: (20000, 7)
[2025-07-06 19:03:16] 📁 Using dataset: (20000, 7)
[2025-07-06 19:03:16] 🔄 Data split:
   Training context: (8000, 7) (samples 0 to 7999)
   Prediction range: (12000, 7) (samples 8000 to 19999)
[2025-07-06 19:03:16] ⚡ Binary flag analysis:
   Total '1' flags: 200
   Pattern check - first 10 indices: [ 99 199 299 399 499 599 699 799 899 999]
   Expected pattern: every 100 steps starting from some offset
   Intervals between flags: [100 100 100 100 100 100 100 100 100 100] (should be mostly 100)

[2025-07-06 19:03:16] 🚀 Starting corrected inference...
[2025-07-06 19:03:17] ✅ Model loaded: (None, 20, 7) → (None, 6)

[2025-07-06 19:03:17] 🎯 Starting horizon 1
[2025-07-06 19:03:17] 🔄 Initial sequence shape: (1, 20, 7)
   Starting fr

[2025-07-06 19:34:21] Step 12000/12000 | Pos: 19999 | MAPE: 3.5466 | Flag: 1.0 | Mode: Auto-Regressive

[2025-07-06 19:34:22] ✅ Horizon 5 completed:
   Average MAPE: 3.5466
   Valid predictions: 12000/12000

[2025-07-06 19:34:22] 🎯 Starting horizon 10
[2025-07-06 19:34:22] 🔄 Initial sequence shape: (1, 20, 7)
   Starting from training samples 7980 to 7999
[2025-07-06 19:34:22] Step     1/12000 | Pos:  8000 | MAPE: 0.0284 | Flag: 0.0 | Mode: Ground Truth Reset
[2025-07-06 19:34:22] Step     2/12000 | Pos:  8001 | MAPE: 0.0312 | Flag: 0.0 | Mode: Auto-Regressive
[2025-07-06 19:34:22] Step     3/12000 | Pos:  8002 | MAPE: 0.0323 | Flag: 0.0 | Mode: Auto-Regressive
[2025-07-06 19:34:22] Step     4/12000 | Pos:  8003 | MAPE: 0.0381 | Flag: 0.0 | Mode: Auto-Regressive
[2025-07-06 19:34:22] Step     5/12000 | Pos:  8004 | MAPE: 0.0337 | Flag: 0.0 | Mode: Auto-Regressive
[2025-07-06 19:34:22] Step     6/12000 | Pos:  8005 | MAPE: 0.0328 | Flag: 0.0 | Mode: Auto-Regressive
[2025-07-06 19:34:22]

In [1]:
import numpy as np
import tensorflow as tf
from keras.models import load_model
import pandas as pd
import time

def timestamp(): 
    return time.strftime('%Y-%m-%d %H:%M:%S')

# ============================================================================
# CORRECTED PARAMETERS AND DATA HANDLING
# ============================================================================

# Key parameters
TRAINING_SAMPLES = 8000      # Samples used for training
TOTAL_SAMPLES = 20000        # Total samples in dataset
seq_length = 20
steps_to_pred = 12000        # How many steps to predict beyond training
batch_size = 32
molecules = 2
species = '2_22'
horizons = np.array([50, 100])

print(f"[{timestamp()}] 🔧 CORRECTED INFERENCE SETUP:")
print(f"   Training used: {TRAINING_SAMPLES} samples")
print(f"   Total dataset: {TOTAL_SAMPLES} samples") 
print(f"   Predicting: {steps_to_pred} steps beyond training")
print(f"   Available for prediction: {TOTAL_SAMPLES - TRAINING_SAMPLES} = {TOTAL_SAMPLES - TRAINING_SAMPLES} samples")

# ============================================================================
# LOAD AND VERIFY DATA
# ============================================================================

# Load the FULL dataset
full_data = np.load(f'{species}_data.npy')
print(f"[{timestamp()}] 📁 Loaded full dataset: {full_data.shape}")

# Verify we have enough data
if TOTAL_SAMPLES > len(full_data):
    print(f"❌ Error: Dataset only has {len(full_data)} samples, need {TOTAL_SAMPLES}")
    exit(1)

if TRAINING_SAMPLES + steps_to_pred > TOTAL_SAMPLES:
    print(f"❌ Error: Training ({TRAINING_SAMPLES}) + prediction ({steps_to_pred}) > total ({TOTAL_SAMPLES})")
    exit(1)

# Use only the specified amount
full_data = full_data[:TOTAL_SAMPLES, :]
print(f"[{timestamp()}] 📁 Using dataset: {full_data.shape}")

# Split data correctly
training_data = full_data[:TRAINING_SAMPLES, :]          # First 8,000 for training context
prediction_data = full_data[TRAINING_SAMPLES:, :]        # Remaining for prediction/validation

print(f"[{timestamp()}] 🔄 Data split:")
print(f"   Training context: {training_data.shape} (samples 0 to {TRAINING_SAMPLES-1})")
print(f"   Prediction range: {prediction_data.shape} (samples {TRAINING_SAMPLES} to {TOTAL_SAMPLES-1})")

# Verify binary flags in full dataset
binary_flags = full_data[:, 6]
ones_indices = np.where(binary_flags == 1)[0]
print(f"[{timestamp()}] ⚡ Binary flag analysis:")
print(f"   Total '1' flags: {len(ones_indices)}")
print(f"   Pattern check - first 10 indices: {ones_indices[:10]}")
print(f"   Expected pattern: every 100 steps starting from some offset")

# Check if pattern is every 100 steps
if len(ones_indices) > 1:
    differences = np.diff(ones_indices)
    print(f"   Intervals between flags: {differences[:10]} (should be mostly 100)")

# ============================================================================
# CORRECTED INFERENCE SETUP
# ============================================================================

def create_initial_sequence_from_training(training_data, seq_length):
    """Create initial sequence from the LAST part of training data"""
    if len(training_data) < seq_length:
        raise ValueError(f"Training data too short: {len(training_data)} < {seq_length}")
    
    # Take the last seq_length samples from training
    sequence = training_data[-seq_length:, :]
    return np.expand_dims(sequence, axis=0)

def get_correct_binary_flag(full_data, training_samples, prediction_step):
    """Get the correct binary flag for a given prediction step"""
    # The actual position in the full dataset
    actual_position = training_samples + prediction_step
    
    if actual_position >= len(full_data):
        # If we've gone beyond available data, extrapolate the pattern
        # Assuming the pattern repeats every 100 steps
        ones_indices = np.where(full_data[:, 6] == 1)[0]
        if len(ones_indices) > 0:
            # Find the pattern
            pattern_start = ones_indices[0] % 100
            flag_value = 1 if (actual_position % 100) == pattern_start else 0
            print(f"   ⚡ Extrapolating flag for position {actual_position}: {flag_value}")
            return flag_value
        else:
            return 0
    
    return full_data[actual_position, 6]

def get_ground_truth_if_available(full_data, training_samples, prediction_step):
    """Get ground truth if available, otherwise return None"""
    actual_position = training_samples + prediction_step
    
    if actual_position < len(full_data):
        return full_data[actual_position, :6].reshape(1, -1)
    else:
        return None

# ============================================================================
# CORRECTED INFERENCE LOOP
# ============================================================================

print(f"\n[{timestamp()}] 🚀 Starting corrected inference...")

# Load model
model = load_model(f'transformer_model_384_{species}.h5')
print(f"[{timestamp()}] ✅ Model loaded: {model.input_shape} → {model.output_shape}")

# Track performance summary
summary = {}

for j, horizon in enumerate(horizons):
    horizon_start_time = time.time()
    
    print(f"\n[{timestamp()}] 🎯 Starting horizon {horizon}")
    print("="*80)
    
    # Initialize tracking
    performance = np.zeros((steps_to_pred, 1))
    predictions = np.zeros((steps_to_pred, 2 * molecules * 3))
    
    # Start with the last sequence from training data
    current_sequence = create_initial_sequence_from_training(training_data, seq_length)
    print(f"[{timestamp()}] 🔄 Initial sequence shape: {current_sequence.shape}")
    print(f"   Starting from training samples {TRAINING_SAMPLES-seq_length} to {TRAINING_SAMPLES-1}")
    
    using_ground_truth = True
    
    for i in range(steps_to_pred):
        # Make prediction
        prediction = model.predict(current_sequence, steps=1, verbose=0)
        
        # Get ground truth if available
        ground_truth = get_ground_truth_if_available(full_data, TRAINING_SAMPLES, i)
        
        # Calculate performance if ground truth available
        if ground_truth is not None:
            error = tf.metrics.mean_absolute_percentage_error(ground_truth, prediction)
            performance[i, 0] = error.numpy()
            predictions[i, :molecules*3] = prediction.flatten()
            predictions[i, molecules*3:] = ground_truth.flatten()
        else:
            performance[i, 0] = -1  # No ground truth available
            predictions[i, :molecules*3] = prediction.flatten()
            if i == 0:
                print(f"[{timestamp()}] ⚠️  No more ground truth available from step {i}")
        
        # Get the correct binary flag
        binary_flag = get_correct_binary_flag(full_data, TRAINING_SAMPLES, i)
        
        # Create full prediction (coordinates + binary flag)
        full_prediction = np.zeros((1, 7))
        full_prediction[0, :6] = prediction.flatten()
        full_prediction[0, 6] = binary_flag
        
        # Update sequence based on horizon strategy
        if horizon == 1:
            # Always use ground truth if available, otherwise use prediction
            if ground_truth is not None:
                next_step = np.zeros((1, 7))
                next_step[0, :6] = ground_truth.flatten()
                next_step[0, 6] = binary_flag
                current_sequence = np.concatenate([current_sequence[:, 1:, :], 
                                                 next_step.reshape(1, 1, -1)], axis=1)
                mode = "Ground Truth"
            else:
                current_sequence = np.concatenate([current_sequence[:, 1:, :], 
                                                 full_prediction.reshape(1, 1, -1)], axis=1)
                mode = "Auto-Regressive"
                using_ground_truth = False
                
        else:
            # Multi-step horizon strategy
            if i % horizon == 0 and ground_truth is not None:
                # Reset with ground truth every 'horizon' steps
                reset_start = max(0, TRAINING_SAMPLES + i - seq_length + 1)
                reset_end = TRAINING_SAMPLES + i + 1
                
                if reset_end <= len(full_data):
                    current_sequence = full_data[reset_start:reset_end, :].reshape(1, seq_length, -1)
                    mode = "Ground Truth Reset"
                    using_ground_truth = True
                else:
                    # Continue with prediction
                    current_sequence = np.concatenate([current_sequence[:, 1:, :], 
                                                     full_prediction.reshape(1, 1, -1)], axis=1)
                    mode = "Auto-Regressive"
                    using_ground_truth = False
            else:
                # Continue with previous prediction
                current_sequence = np.concatenate([current_sequence[:, 1:, :], 
                                                 full_prediction.reshape(1, 1, -1)], axis=1)
                mode = "Auto-Regressive"
                using_ground_truth = False
        
        # Progress reporting
        if (i + 1) % 500 == 0 or i < 10:
            actual_pos = TRAINING_SAMPLES + i
            valid_perf = performance[:i+1][performance[:i+1, 0] >= 0, 0]
            
            if len(valid_perf) > 0:
                avg_mape = np.mean(valid_perf)
                print(f"[{timestamp()}] Step {i+1:5d}/{steps_to_pred} | Pos: {actual_pos:5d} | "
                      f"MAPE: {avg_mape:.4f} | Flag: {binary_flag} | Mode: {mode}")
            else:
                print(f"[{timestamp()}] Step {i+1:5d}/{steps_to_pred} | Pos: {actual_pos:5d} | "
                      f"No valid MAPE | Flag: {binary_flag} | Mode: {mode}")
    
    # Save results
    perf_file = f"corrected_performance_{species}_horizon_{horizon}.csv"
    pred_file = f"corrected_predictions_{species}_horizon_{horizon}.csv"
    np.savetxt(perf_file, performance, delimiter=",")
    np.savetxt(pred_file, predictions, delimiter=",")
    
    # Calculate final statistics
    valid_perf = performance[performance[:, 0] >= 0, 0]
    if len(valid_perf) > 0:
        avg_perf = np.mean(valid_perf)
        print(f"\n[{timestamp()}] ✅ Horizon {horizon} completed:")
        print(f"   Average MAPE: {avg_perf:.4f}")
        print(f"   Valid predictions: {len(valid_perf)}/{steps_to_pred}")
        summary[horizon] = avg_perf
    else:
        print(f"\n[{timestamp()}] ❌ Horizon {horizon}: No valid predictions")
        summary[horizon] = None

# Final summary
print(f"\n{'='*80}")
print(f"[{timestamp()}] 🎉 CORRECTED INFERENCE COMPLETED")
print(f"{'='*80}")
for h, v in summary.items():
    if v is not None:
        print(f"Horizon {h:4d}: {v:.4f} MAPE")
    else:
        print(f"Horizon {h:4d}: No valid data")

print(f"\n[{timestamp()}] 💡 Key corrections made:")
print(f"   ✅ Start prediction from end of training data (sample {TRAINING_SAMPLES})")
print(f"   ✅ Use correct binary flags from proper temporal positions")
print(f"   ✅ Handle ground truth availability correctly")
print(f"   ✅ Maintain proper sequence continuity")


[2025-08-14 22:25:45] 🔧 CORRECTED INFERENCE SETUP:
   Training used: 8000 samples
   Total dataset: 20000 samples
   Predicting: 12000 steps beyond training
   Available for prediction: 12000 = 12000 samples
[2025-08-14 22:25:45] 📁 Loaded full dataset: (20000, 7)
[2025-08-14 22:25:45] 📁 Using dataset: (20000, 7)
[2025-08-14 22:25:45] 🔄 Data split:
   Training context: (8000, 7) (samples 0 to 7999)
   Prediction range: (12000, 7) (samples 8000 to 19999)
[2025-08-14 22:25:45] ⚡ Binary flag analysis:
   Total '1' flags: 200
   Pattern check - first 10 indices: [ 99 199 299 399 499 599 699 799 899 999]
   Expected pattern: every 100 steps starting from some offset
   Intervals between flags: [100 100 100 100 100 100 100 100 100 100] (should be mostly 100)

[2025-08-14 22:25:45] 🚀 Starting corrected inference...

[2025-08-14 22:25:52] ✅ Model loaded: (None, 20, 7) → (None, 6)

[2025-08-14 22:25:52] 🎯 Starting horizon 50
[2025-08-14 22:25:52] 🔄 Initial sequence shape: (1, 20, 7)
   Starting

[2025-08-14 23:25:14] Step  9000/12000 | Pos: 16999 | MAPE: 3.9342 | Flag: 1.0 | Mode: Auto-Regressive
[2025-08-14 23:26:11] Step  9500/12000 | Pos: 17499 | MAPE: 4.1808 | Flag: 1.0 | Mode: Auto-Regressive
[2025-08-14 23:27:07] Step 10000/12000 | Pos: 17999 | MAPE: 4.4207 | Flag: 1.0 | Mode: Auto-Regressive
[2025-08-14 23:28:03] Step 10500/12000 | Pos: 18499 | MAPE: 4.6238 | Flag: 1.0 | Mode: Auto-Regressive
[2025-08-14 23:29:11] Step 11000/12000 | Pos: 18999 | MAPE: 4.7955 | Flag: 1.0 | Mode: Auto-Regressive
[2025-08-14 23:30:47] Step 11500/12000 | Pos: 19499 | MAPE: 5.0184 | Flag: 1.0 | Mode: Auto-Regressive
[2025-08-14 23:32:13] Step 12000/12000 | Pos: 19999 | MAPE: 5.2475 | Flag: 1.0 | Mode: Auto-Regressive

[2025-08-14 23:32:14] ✅ Horizon 100 completed:
   Average MAPE: 5.2475
   Valid predictions: 12000/12000

[2025-08-14 23:32:14] 🎉 CORRECTED INFERENCE COMPLETED
Horizon   50: 5.5260 MAPE
Horizon  100: 5.2475 MAPE

[2025-08-14 23:32:14] 💡 Key corrections made:
   ✅ Start predicti